In [1]:
import re

import Models.JournalModel as JM
import pandas as pd

ej = JM.Journal("ej")

In [30]:
#cambridge = JM.Journal("cje")

#cambridge.volume(31).issue(4).article(8).table(5).footnotes # double introduction



None
None
None
None
None
None
None
None


AttributeError: 'NoneType' object has no attribute 'findAll'

In [2]:

def scan_article(a):
    fail_list = []
    output_df = pd.DataFrame()
    table_id = 1
    headings = a.list_headings('list')

    # check if the word placebo is found anywhere in the article
    placebo_article = len(re.findall(r'placebo tests*', a.soup.text))

    for h in headings:
        if h.name.lower() in ['notes', 'appendix', 'references', 'supplementary data', 'author notes']:
            continue

        # check if the word placebo is found in the heading
        placebo_section = len(re.findall(r'placebo tests*', h.text))

        for t in h.list_tables('list'):
            try:
                # get the coefficients
                cdf = t.list_coefficients()

                if isinstance(cdf, pd.DataFrame):

                    # check if word placebo in table
                    table_string = t.df.to_string() + t.footnotes
                    placebo_table = len(re.findall(r'placebo tests*', table_string))

                    # specify columns to be appended
                    cols_to_add = [
                        ('journal', a.journal().abbreviation),
                        ('volume', a.volume_number),
                        ('year', a.year),
                        ('issue', a.issue_number),
                        ('article_id', a.id),
                        ('article_doi', a.doi),
                        ('placebo_article', placebo_article),
                        ('heading_id', h.id_global),
                        ('heading_name', h.name),
                        ('placebo_section', placebo_section),
                        ('table_id', table_id ),
                        ('table_name', t.name),
                        ('placebo_table', placebo_table)

                    ]

                    # now append the columns to the df
                    for c in cols_to_add:
                        cdf[c[0]] = c[1]
                    col_names_to_add = [x[0] for x in cols_to_add]
                    existing_cols = list(cdf.columns)
                    cdf = cdf.reindex(columns=(col_names_to_add + existing_cols))

                    # add the coefficient df to the output df
                    if output_df.empty:
                        output_df = cdf
                    else:
                        output_df = output_df.append(cdf)
            except:
                # table could not be analyzed
                fail_list.append(table_id)

            table_id += 1

    return (output_df, fail_list)

#a = ej.find_article_by_doi("https://doi.org/10.1111/ecoj.12509")
#scan_article(a)


In [3]:
class fail_tracker:
    def __init__(self):
        self.dic = {'volume': [], 'issue': [], 'article': [], 'failed_all': [], 'tables': []}

    def insert(self, v, i, a, fa, t):
        self.dic['volume'].append(v)
        self.dic['issue'].append(i)
        self.dic['article'].append(a)
        self.dic['failed_all'].append(fa)
        self.dic['tables'].append(t)

    def fail_article(self, a, verbose=False):
        self.insert(a.volume_number, a.issue_number, a.doi, 1, None)
        if verbose:
            print(f"Failed article {a.id}.")

    def fail_tables(self, a, failed_tables, verbose=False):
        self.insert(a.volume_number, a.issue_number, a.doi, 0, failed_tables)
        if verbose:
            print(f"Completed article {a.id}: Failed tables {failed_tables}.")

    def compile_df(self):
        return pd.DataFrame(self.dic)

In [5]:
# study PLACEBO

# vol = ej.volume(128)
# iss = vol.issue(614)
# a = iss.article(6)
# tables = a.list_tables("list")
# for t in tables:
#     print(t.list_coefficients())


In [5]:
# all articles from a journal
def draw_random_articles(journal, n, exclude_doi=[]):
    df = journal.ref_df
    shuffled_df = df.sample(n=len(df), random_state = 23500)
    filtered_df = shuffled_df.loc[shuffled_df['doi'].isin(exclude_doi), :]
    output = filtered_df.iloc[0:n,:]
    return output

# dois from Sonja's tests
sonja_dois = pd.read_excel('Spreadsheets/sonja_evaluation_rounds.xlsx')['article'].values

# our new ej sample
ej_sample = draw_random_articles(ej, 100, sonja_dois)['doi'].values



In [6]:
# scan all of the articles in EJ
ft = fail_tracker()
failed_tables = []
output_df = pd.DataFrame()

# Loop volumes
#for v in ej.list_volumes('list')[0:1]:
#    vol = ej.volume(v)
#
#    # Loop issues
#    for i in vol.list_issues('list')[0:1]:
#        iss = vol.issue(i)
#
#        # Loop articles
#        for a in iss.list_articles('list'):

articles = [ej.find_article_by_doi(x) for x in ej_sample]

for doi in ej_sample:
    a = ej.find_article_by_doi(doi)
    try:
        x = scan_article(a)
    except:
        ft.fail_article(a, verbose=True)
        continue
    df = x[0] # extract the returned df of coefficients
    failed_tables = x[1]  # extract the list of failed tables

    # append the coefficient df to the output_df
    if df.empty == False:
        if output_df.empty:
            output_df = df
        else:
            output_df = output_df.append(df)

    if len(failed_tables) == 0:
        print(f"Completed article {a.id}")
    else:
        ft.fail_tables(a, failed_tables, True)


output_df



Completed article 11
Completed article 9
Completed article 2
Completed article 22
Completed article 2
Completed article 7
Completed article 6
Completed article 4
Completed article 4
Completed article 5
Completed article 13
Failed article 2.
Completed article 11
Completed article 3
Completed article 2
Completed article 11
Completed article 1
Completed article 6
Completed article 7: Failed tables [2, 3].
Completed article 1
Completed article 12: Failed tables [2, 4].
Completed article 7
Failed article 11.
Completed article 5
Failed article 2.
Completed article 1
Completed article 7
Completed article 6
Completed article 4
Completed article 13
Completed article 10
Completed article 2
Completed article 3
Completed article 7
Completed article 1: Failed tables [6].
Completed article 3
Completed article 8
Failed article 9.
Completed article 12
Completed article 8
Completed article 12: Failed tables [2].
Failed article 7.
Completed article 12
Completed article 10
Completed article 8
Failed arti

,journal,volume,year,issue,article_id,article_doi,placebo_article,heading_id,heading_name,placebo_section,...,issue,article_id,article_doi,placebo_article,heading_id,heading_name,placebo_section,table_id,table_name,placebo_table
0,ej,115,2005,501,2,https://doi.org/10.1111/j.0013-0133.2005.00970.x,0,6,5. Results,0,...,501,2,https://doi.org/10.1111/j.0013-0133.2005.00970.x,0,6,5. Results,0,3,Table 3,0
1,ej,115,2005,501,2,https://doi.org/10.1111/j.0013-0133.2005.00970.x,0,6,5. Results,0,...,501,2,https://doi.org/10.1111/j.0013-0133.2005.00970.x,0,6,5. Results,0,3,Table 3,0
2,ej,115,2005,501,2,https://doi.org/10.1111/j.0013-0133.2005.00970.x,0,6,5. Results,0,...,501,2,https://doi.org/10.1111/j.0013-0133.2005.00970.x,0,6,5. Results,0,3,Table 3,0
0,ej,115,2005,501,2,https://doi.org/10.1111/j.0013-0133.2005.00970.x,0,6,5. Results,0,...,501,2,https://doi.org/10.1111/j.0013-0133.2005.00970.x,0,6,5. Results,0,4,Table 4,0
1,ej,115,2005,501,2,https://doi.org/10.1111/j.0013-0133.2005.00970.x,0,6,5. Results,0,...,501,2,https://doi.org/10.1111/j.0013-0133.2005.00970.x,0,6,5. Results,0,4,Table 4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,ej,115,2005,507,4,https://doi.org/10.1111/j.1468-0297.2005.01039.x,0,5,4. Conclusion,0,...,507,4,https://doi.org/10.1111/j.1468-0297.2005.01039.x,0,5,4. Conclusion,0,8,None,0
62,ej,115,2005,507,4,https://doi.org/10.1111/j.1468-0297.2005.01039.x,0,5,4. Conclusion,0,...,507,4,https://doi.org/10.1111/j.1468-0297.2005.01039.x,0,5,4. Conclusion,0,8,None,0
63,ej,115,2005,507,4,https://doi.org/10.1111/j.1468-0297.2005.01039.x,0,5,4. Conclusion,0,...,507,4,https://doi.org/10.1111/j.1468-0297.2005.01039.x,0,5,4. Conclusion,0,8,None,0
64,ej,115,2005,507,4,https://doi.org/10.1111/j.1468-0297.2005.01039.x,0,5,4. Conclusion,0,...,507,4,https://doi.org/10.1111/j.1468-0297.2005.01039.x,0,5,4. Conclusion,0,8,None,0


In [7]:
# export the output to excel
writer = pd.ExcelWriter('output.xlsx',engine='xlsxwriter',options={'strings_to_urls': False})
output_df.to_excel(writer, index=False)
writer.close()


In [8]:
# export the fail list to excel
writer = pd.ExcelWriter('fails.xlsx',engine='xlsxwriter',options={'strings_to_urls': False})
ft.compile_df().to_excel(writer, index=False)
writer.close()

"One concern of feminist economists is with the appropriateness of mainstream methodology for understanding social reality (England, 1993; Folbre, 1993; Humphries, 1995; Lawson, 1997). This concern arises because of the limitations of a categorical structure of analysis put in place by mainstream economists, and the partial orientation of this analytical structure. The dominant paradigm is absolutist in nature, mostly involving isolated individuals operating in conditions where an equilibrium outcome is the result, which is often fictitious and is a way of providing universalising accounts.1 Feminist economists point out that this feature is motivated by the reduction of human relationships to one dominant Western white male ideal as a stereotype (England, 1993; Folbre, 1993; Humphries, 1995). Others argue that this outcome is brought about by a desire to facilitate the universal application of analytic or formalistic methods within economics (Lawson, 1997, 1999). Certainly, to underst